<a href="https://colab.research.google.com/github/yohei0408/kikagaku/blob/main/20241009_signateCup_%E5%AE%9F%E8%B7%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 必要なモジュールのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


plt.style.use('seaborn')

# 警告の無視
import warnings
warnings.filterwarnings('ignore')

<ipython-input-1-019144cfb2a2>:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [3]:
df = pd.read_csv('/content/df_train_3489.csv')
df = df.drop(['Unnamed: 0','id'], axis=1)
df.head()

,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,customer_info,ProdTaken,結婚,車,子供の数
0,50,0.11610,2,900,0.224551,0.165172,1.0,4.0,0.297143,3.0,5,1,4,0.302392,253905,未婚 車未所持 子供なし,1,0.205699,0.153927,0
1,56,0.17907,1,840,0.098926,0.165172,1.0,4.0,0.089286,3.0,2,1,4,0.086595,404475,"離婚済み,車あり,子供無し",0,0.116193,0.116883,0
2,30,0.11610,1,600,0.224551,0.100686,1.0,3.0,0.297143,3.0,4,0,4,0.302392,278145,"結婚済み、自動車未所有,子供なし",1,0.094192,0.153927,0
3,20,0.11610,2,1080,0.153699,0.100686,1.0,3.0,0.089286,4.0,1,0,5,0.086595,326805,離婚済み、車所持、子供無し,0,0.116193,0.116883,0
4,48,0.17907,3,1020,0.153699,0.100686,1.0,3.0,0.297143,4.0,4,0,4,0.302392,258435,独身／車所持／無子,1,0.205699,0.116883,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     3489 non-null   int64  
 1   TypeofContact           3489 non-null   float64
 2   CityTier                3489 non-null   int64  
 3   DurationOfPitch         3489 non-null   int64  
 4   Occupation              3489 non-null   float64
 5   Gender                  3489 non-null   float64
 6   NumberOfPersonVisiting  3489 non-null   float64
 7   NumberOfFollowups       3489 non-null   float64
 8   ProductPitched          3489 non-null   float64
 9   PreferredPropertyStar   3489 non-null   float64
 10  NumberOfTrips           3489 non-null   int64  
 11  Passport                3489 non-null   int64  
 12  PitchSatisfactionScore  3489 non-null   int64  
 13  Designation             3489 non-null   float64
 14  MonthlyIncome           3489 non-null   

In [5]:
df = df.drop('customer_info', axis=1)
df.head()

,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,結婚,車,子供の数
0,50,0.11610,2,900,0.224551,0.165172,1.0,4.0,0.297143,3.0,5,1,4,0.302392,253905,1,0.205699,0.153927,0
1,56,0.17907,1,840,0.098926,0.165172,1.0,4.0,0.089286,3.0,2,1,4,0.086595,404475,0,0.116193,0.116883,0
2,30,0.11610,1,600,0.224551,0.100686,1.0,3.0,0.297143,3.0,4,0,4,0.302392,278145,1,0.094192,0.153927,0
3,20,0.11610,2,1080,0.153699,0.100686,1.0,3.0,0.089286,4.0,1,0,5,0.086595,326805,0,0.116193,0.116883,0
4,48,0.17907,3,1020,0.153699,0.100686,1.0,3.0,0.297143,4.0,4,0,4,0.302392,258435,1,0.205699,0.116883,0


In [6]:
# ランダムフォレストとAUC評価指標をインポートします。
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
# scikit-learnからtrain_test_splitをインポートします。
from sklearn.model_selection import train_test_split

In [7]:
x = df.drop(['ProdTaken'], axis=1)
t = df['ProdTaken']
from sklearn import metrics
# DataFrame 型から values で数値を取得する
x = x.values
t = t.values
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=0)

from xgboost import XGBClassifier
# モデルの宣言
xgb = XGBClassifier(random_state=0, n_estimators=25,learning_rate=0.3, booster='gbtree',max_depth=6,
                    min_child_weight=10, gamma=0.1, scale_pos_weight=1)
# モデルの学習
xgb.fit(x_train, t_train)
# モデルの検証
print(xgb.score(x_train, t_train))
# 推論
y_pred = xgb.predict(x_test)
# AccuracyとRecallの確認
print('accuracy : ', metrics.accuracy_score(t_test, y_pred))
print('recall : ', metrics.recall_score(t_test, y_pred, average=None))
print('f1_score :', metrics.f1_score(t_test, y_pred, average=None))
from sklearn.metrics import roc_auc_score
# AUCの評価はroc_auc_score(正解データ, 予測データ)で可能です。
# 成約したかどうかを判定するため、2列目の1の確率(成約している確率)を予測データとして入力します。
pred = xgb.predict_proba(x_test)
score = roc_auc_score(t_test, pred[:, 1])
print(score)

0.9098726114649681
accuracy :  0.8595988538681948
recall :  [0.98293515 0.21428571]
f1_score : [0.9216     0.32876712]
0.7918088737201365


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# 1. データの分割
# X_train, X_test, y_train, y_test は事前に用意されていると仮定

# 2. 使用するモデルのリスト
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "XGB" : XGBClassifier()
}

# 3. 結果を保存するためのリスト
results = []

# 4. 各モデルのトレーニングと評価
for model_name, model in models.items():
    # モデルの学習
    model.fit(x_train, t_train)

    # テストデータに対する予測
    y_pred = model.predict(x_test)

    # 正解率とF1スコアの計算
    accuracy = accuracy_score(t_test, y_pred)
    f1 = f1_score(t_test, y_pred)

    # 結果をリストに追加
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "F1 Score": f1
    })

# 5. 結果をDataFrameに変換
results_df = pd.DataFrame(results)

# 6. 結果を表示
print(results_df)


                 Model  Accuracy  F1 Score
0  Logistic Regression  0.839542  0.000000
1        Random Forest  0.848138  0.253521
2        Decision Tree  0.779370  0.237624
3  K-Nearest Neighbors  0.816619  0.085714
4          Naive Bayes  0.842407  0.035088
5                  XGB  0.839542  0.300000


In [9]:
test = pd.read_csv('/content/df_test.csv')
test = test.drop('Unnamed: 0', axis=1)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      3489 non-null   int64  
 1   Age                     3489 non-null   int64  
 2   TypeofContact           3489 non-null   float64
 3   CityTier                3489 non-null   int64  
 4   DurationOfPitch         3489 non-null   int64  
 5   Occupation              3489 non-null   float64
 6   Gender                  3489 non-null   float64
 7   NumberOfPersonVisiting  3489 non-null   float64
 8   NumberOfFollowups       3489 non-null   float64
 9   ProductPitched          3489 non-null   float64
 10  PreferredPropertyStar   3489 non-null   float64
 11  NumberOfTrips           3489 non-null   int64  
 12  Passport                3489 non-null   int64  
 13  PitchSatisfactionScore  3489 non-null   int64  
 14  Designation             3489 non-null   

In [10]:
test_x = test.drop([ 'customer_info', 'id'], axis=1)
from sklearn import metrics
# DataFrame 型から values で数値を取得する
test_x = test_x.values


In [11]:
# 予測の確率を取得
y_proba = xgb.predict_proba(test_x)
y = (y_proba[:,1] > 0.5).astype('i')

zero_count = (y == 0).sum()

# 結果を表示
print(f"y列の0の数: {zero_count}")
result_df = pd.DataFrame({
    'id': test['id'],  # 1列目: test_dfのid列
    'prediction': y       # 2列目: 予測値y
})
result_df = result_df.astype(int)
result_df.head()

y列の0の数: 3241


,id,prediction
0,3489,0
1,3490,0
2,3491,0
3,3492,0
4,3493,0


In [12]:
# 列名なしでCSVに保存
result_df.to_csv('submit_xgb.csv', index=False, header=False)

In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      3489 non-null   int64  
 1   Age                     3489 non-null   int64  
 2   TypeofContact           3489 non-null   float64
 3   CityTier                3489 non-null   int64  
 4   DurationOfPitch         3489 non-null   int64  
 5   Occupation              3489 non-null   float64
 6   Gender                  3489 non-null   float64
 7   NumberOfPersonVisiting  3489 non-null   float64
 8   NumberOfFollowups       3489 non-null   float64
 9   ProductPitched          3489 non-null   float64
 10  PreferredPropertyStar   3489 non-null   float64
 11  NumberOfTrips           3489 non-null   int64  
 12  Passport                3489 non-null   int64  
 13  PitchSatisfactionScore  3489 non-null   int64  
 14  Designation             3489 non-null   

In [14]:
import pandas as pd
df10 = df.copy()
# サンプルデータフレーム
# df = pd.DataFrame(...)  # 実際のデータフレームを使用

# loan_statusが1のサンプル数と0のサンプル数を分ける
df_ProdTaken_1 = df10[df10['ProdTaken'] == 1]
df_ProdTaken_0 = df10[df10['ProdTaken'] == 0]

# どちらか少ないほうのサンプル数を取得
min_samples = min(len(df_ProdTaken_1), len(df_ProdTaken_0))

# 多い方のクラスを少数クラスと同じ数だけランダムにサンプリング
df_ProdTaken_1_downsampled = df_ProdTaken_1.sample(n=min_samples, random_state=0)
df_ProdTaken_0_downsampled = df_ProdTaken_0.sample(n=min_samples, random_state=0)

# ダウンサンプリング後のデータを結合
df_11 = pd.concat([df_ProdTaken_1_downsampled, df_ProdTaken_0_downsampled])

# データをシャッフル（任意）
df_11 = df_11.sample(frac=1, random_state=0).reset_index(drop=True)

# 結果を確認
print(df_11['ProdTaken'].value_counts())

ProdTaken
0    497
1    497
Name: count, dtype: int64


In [15]:
x = df_11.drop(['ProdTaken'], axis=1)
t = df_11['ProdTaken']
from sklearn import metrics
# DataFrame 型から values で数値を取得する
x = x.values
t = t.values
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, random_state=0)

from xgboost import XGBClassifier
# モデルの宣言
xgb = XGBClassifier(random_state=0, n_estimators=25,learning_rate=0.3, booster='gbtree',max_depth=6,
                    min_child_weight=10, gamma=0.1, scale_pos_weight=1)
# モデルの学習
xgb.fit(x_train, t_train)
# モデルの検証
print(xgb.score(x_train, t_train))
# 推論
y_pred = xgb.predict(x_test)
# AccuracyとRecallの確認
print('accuracy : ', metrics.accuracy_score(t_test, y_pred))
print('recall : ', metrics.recall_score(t_test, y_pred, average=None))
print('f1_score :', metrics.f1_score(t_test, y_pred, average=None))
from sklearn.metrics import roc_auc_score
# AUCの評価はroc_auc_score(正解データ, 予測データ)で可能です。
# 成約したかどうかを判定するため、2列目の1の確率(成約している確率)を予測データとして入力します。
pred = xgb.predict_proba(x_test)
score = roc_auc_score(t_test, pred[:, 1])
print(score)

0.8474820143884892
accuracy :  0.7558528428093646
recall :  [0.75735294 0.75460123]
f1_score : [0.73835125 0.77115987]
0.8431974016600506


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb

# 1. データの分割
# X_train, X_test, y_train, y_test は事前に用意されていると仮定

# 2. 使用するモデルのリスト
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "XGB" : XGBClassifier(),
    "LightGBM" : lgb.LGBMClassifier()
}

# 3. 結果を保存するためのリスト
results = []

# 4. 各モデルのトレーニングと評価
for model_name, model in models.items():
    # モデルの学習
    model.fit(x_train, t_train)

    # テストデータに対する予測
    y_pred = model.predict(x_test)

    # 正解率とF1スコアの計算
    accuracy = accuracy_score(t_test, y_pred)
    f1 = f1_score(t_test, y_pred)

    # 結果をリストに追加
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "F1 Score": f1
    })

# 5. 結果をDataFrameに変換
results_df = pd.DataFrame(results)

# 6. 結果を表示
print(results_df)


[LightGBM] [Info] Number of positive: 334, number of negative: 361
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001440 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 695, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.480576 -> initscore=-0.077737
[LightGBM] [Info] Start training from score -0.077737
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [18]:
x = df_11.drop(['ProdTaken','DurationOfPitch','Designation'], axis=1)
t = df_11['ProdTaken']
from sklearn import metrics
# DataFrame 型から values で数値を取得する
x = x.values
t = t.values
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, random_state=0)

from xgboost import XGBClassifier
# モデルの宣言
model = RandomForestClassifier(
    n_estimators=150,          # 決定木の数
    max_depth=10,              # 最大深さ
    min_samples_split=5,       # ノードを分割する最小サンプル数
    min_samples_leaf=3,        # 葉ノードに必要な最小サンプル数
    max_features='sqrt',       # 特徴量の選び方
    bootstrap=True,            # ブートストラップを使う
    random_state=42            # 再現性のための乱数シード
)
# モデルの学習
model.fit(x_train, t_train)
# モデルの検証
print(model.score(x_train, t_train))
# 推論
y_pred = model.predict(x_test)
# AccuracyとRecallの確認
print('accuracy : ', metrics.accuracy_score(t_test, y_pred))
print('recall : ', metrics.recall_score(t_test, y_pred, average=None))
print('f1_score :', metrics.f1_score(t_test, y_pred, average=None))
from sklearn.metrics import roc_auc_score
# AUCの評価はroc_auc_score(正解データ, 予測データ)で可能です。
# 成約したかどうかを判定するため、2列目の1の確率(成約している確率)を予測データとして入力します。
pred = model.predict_proba(x_test)
score = roc_auc_score(t_test, pred[:, 1])
print(score)

0.935251798561151
accuracy :  0.7491638795986622
recall :  [0.77205882 0.73006135]
f1_score : [0.73684211 0.76038339]
0.8466257668711656


In [220]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score

# Dropping unnecessary columns
x = df_11.drop(['ProdTaken', 'NumberOfFollowups', 'NumberOfTrips', 'DurationOfPitch','PreferredPropertyStar', 'PitchSatisfactionScore'], axis=1)
# x = df_11.drop(['ProdTaken'], axis=1)
t = df_11['ProdTaken']

# Convert DataFrame to NumPy arrays
x = x.values
t = t.values

# Split the data into training and test sets
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, random_state=0)

# Initialize LightGBM model with parameters
model = lgb.LGBMClassifier(num_leaves=4,min_data_in_leaf=4,max_depth=-1,
                           learning_rate=0.05,random_state=42,
                            max_bin=15, num_iterations=355,
                           min_sum_hessian_in_leaf=0.6)

# Train the model
model.fit(x_train, t_train)

# Model accuracy on training set
#print(model.score(x_train, t_train))

# Make predictions
y_pred = model.predict(x_test)
y_proba = model.predict_proba(x_test)
y_pred = (y_proba[:,1] > 0.5).astype('i')

# Accuracy, recall, and F1-score
#print('accuracy : ', metrics.accuracy_score(t_test, y_pred))
#print('recall : ', metrics.recall_score(t_test, y_pred, average=None))
print('f1_score :', metrics.f1_score(t_test, y_pred, average=None))

# AUC score
pred = model.predict_proba(x_test)
score = roc_auc_score(t_test, pred[:, 1])
print(score)

[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.6, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.6
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.6, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.6
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Info] Number of positive: 334, number of negative: 361
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000262 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 74
[LightGBM] [Info] Number of data points in the train set: 695, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.480576 -> initscore=-0.077737
[LightGBM] [Info] Start training from score -0.0

In [221]:
test_x = test.drop([ 'customer_info', 'id', 'NumberOfFollowups', 'NumberOfTrips', 'DurationOfPitch','PreferredPropertyStar', 'PitchSatisfactionScore'], axis=1)
from sklearn import metrics
# DataFrame 型から values で数値を取得する
test_x = test_x.values


In [222]:
# 予測の確率を取得
y_proba = model.predict_proba(test_x)
y = (y_proba[:,1] > 0.5).astype('i')

zero_count = (y == 0).sum()

# 結果を表示
print(f"y列の0の数: {zero_count}")
result_df = pd.DataFrame({
    'id': test['id'],  # 1列目: test_dfのid列
    'prediction': y       # 2列目: 予測値y
})
result_df = result_df.astype(int)
result_df.head()

[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.6, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.6
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
y列の0の数: 2459


,id,prediction
0,3489,0
1,3490,1
2,3491,1
3,3492,1
4,3493,1


In [223]:
# 列名なしでCSVに保存
result_df.to_csv('submit_3489_down_lgbm_best.csv', index=False, header=False)

In [75]:
df_test = pd.read_csv('/content/df_test.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              3489 non-null   int64  
 1   id                      3489 non-null   int64  
 2   Age                     3489 non-null   int64  
 3   TypeofContact           3489 non-null   float64
 4   CityTier                3489 non-null   int64  
 5   DurationOfPitch         3489 non-null   int64  
 6   Occupation              3489 non-null   float64
 7   Gender                  3489 non-null   float64
 8   NumberOfPersonVisiting  3489 non-null   float64
 9   NumberOfFollowups       3489 non-null   float64
 10  ProductPitched          3489 non-null   float64
 11  PreferredPropertyStar   3489 non-null   float64
 12  NumberOfTrips           3489 non-null   int64  
 13  Passport                3489 non-null   int64  
 14  PitchSatisfactionScore  3489 non-null   

In [85]:
df_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     994 non-null    int64  
 1   TypeofContact           994 non-null    float64
 2   CityTier                994 non-null    int64  
 3   DurationOfPitch         994 non-null    int64  
 4   Occupation              994 non-null    float64
 5   Gender                  994 non-null    float64
 6   NumberOfPersonVisiting  994 non-null    float64
 7   NumberOfFollowups       994 non-null    float64
 8   ProductPitched          994 non-null    float64
 9   PreferredPropertyStar   994 non-null    float64
 10  NumberOfTrips           994 non-null    int64  
 11  Passport                994 non-null    int64  
 12  PitchSatisfactionScore  994 non-null    int64  
 13  Designation             994 non-null    float64
 14  MonthlyIncome           994 non-null    in

In [86]:
df_11.corr()

,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,結婚,車,子供の数
Age,1.000000,-0.020001,-0.062457,-0.035023,-0.131158,0.010508,0.074762,0.014334,-0.367386,0.150904,0.147051,0.053299,-0.073085,-0.371772,0.446558,-0.250935,-0.193345,-0.035159,0.078470
TypeofContact,-0.020001,1.000000,0.102171,0.014965,-0.027282,-0.082308,-0.186515,-0.065612,0.018851,-0.035318,-0.127886,0.071082,0.014489,0.001308,-0.091705,0.122618,0.089770,0.007069,-0.153794
CityTier,-0.062457,0.102171,1.000000,0.085834,-0.063751,-0.005559,-0.079185,-0.055724,-0.154357,0.035043,-0.078623,0.036506,-0.041416,-0.154298,0.008082,0.110407,0.091949,0.015438,-0.022962
DurationOfPitch,-0.035023,0.014965,0.085834,1.000000,0.051549,-0.043099,-0.041645,-0.057952,0.003623,-0.015757,0.008902,-0.027480,-0.000664,0.005181,-0.002177,0.003334,0.017860,0.039449,-0.045551
Occupation,-0.131158,-0.027282,-0.063751,0.051549,1.000000,-0.035082,-0.047672,0.017267,0.261015,0.086942,0.026191,-0.020347,-0.027201,0.261479,-0.177115,0.158762,0.099911,0.069698,-0.056400
Gender,0.010508,-0.082308,-0.005559,-0.043099,-0.035082,1.000000,0.071883,0.059248,0.073813,-0.051013,0.039962,-0.003230,0.004106,0.060712,0.045004,0.138316,-0.042190,0.083871,0.058156
NumberOfPersonVisiting,0.074762,-0.186515,-0.079185,-0.041645,-0.047672,0.071883,1.000000,0.344323,-0.053015,0.044707,0.164717,-0.059287,-0.081682,-0.036161,0.223756,-0.133974,-0.112253,0.035458,0.687119
NumberOfFollowups,0.014334,-0.065612,-0.055724,-0.057952,0.017267,0.059248,0.344323,1.000000,0.078129,0.001211,0.144050,-0.039084,-0.054696,0.076306,0.165581,0.039915,-0.008198,0.029430,0.338847
ProductPitched,-0.367386,0.018851,-0.154357,0.003623,0.261015,0.073813,-0.053015,0.078129,1.000000,-0.087060,0.023203,0.032289,0.063157,0.975583,-0.529470,0.435609,0.130817,-0.006241,-0.091939
PreferredPropertyStar,0.150904,-0.035318,0.035043,-0.015757,0.086942,-0.051013,0.044707,0.001211,-0.087060,1.000000,-0.014131,-0.031278,0.035459,-0.093136,0.168963,-0.001778,-0.007073,-0.031284,-0.001231


In [87]:
# 相関行列を計算
corr_matrix = df_11.corr()

# ProdTakenとの相関を抽出し、降順にソート
prod_taken_corr = corr_matrix['ProdTaken'].sort_values(ascending=False)

# 表示
print(prod_taken_corr)


ProdTaken                 1.000000
Designation               0.438386
ProductPitched            0.435609
結婚                        0.229448
Passport                  0.219011
Occupation                0.158762
Gender                    0.138316
車                         0.124707
TypeofContact             0.122618
CityTier                  0.110407
NumberOfFollowups         0.039915
NumberOfTrips             0.009960
DurationOfPitch           0.003334
PreferredPropertyStar    -0.001778
PitchSatisfactionScore   -0.018430
子供の数                     -0.118601
NumberOfPersonVisiting   -0.133974
Age                      -0.250935
MonthlyIncome            -0.258184
Name: ProdTaken, dtype: float64
